In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from ipyleaflet import (Map, Marker, basemaps, basemap_to_tiles,
                        FullScreenControl, LayersControl, LayerGroup,
                        WidgetControl, Circle
                       )
from ipywidgets import HTML, Layout, Dropdown, Output, Textarea, VBox, Label
from datamanager import load_wifi_loc_data
import random

def classify_wifi_strength(rssi):
    """
    Categories for WiFi RSSI Measurements
        Excellent: x >= -50 dbm
        Good: -50 dbm > x >= -60 dbm 
        Fair: -60 dbm > x >= -70 dbm
        Weak: -70 dbm x
        
    Parameters:
        rssi:integer
            The RSSI value
        
    Returns:
        The category of the WiFi signal strength
    """
    if rssi >= -50: return 'Excellent'
    elif rssi < -50 and rssi >= -60: return 'Good'
    elif rssi < -60 and rssi >= -70: return 'Fair'
    return 'Weak'

STRENGTH_COLOUR = {
    # Blue for Excellent
    'Excellent' : '#0A75F0',
    # Green for Good
    'Good' : '#0AF01D',
    # Yellow for Fair
    'Fair' : '#FFFB08',
    # Red for Weak
    'Weak' : '#FF0C08'
}

In [3]:
wifi_data = load_wifi_loc_data('whitfeld-3', alternative_folder='old')
wifi_data

,lat,lon,time,bssid,quality,rssi,essid
0,-31.976884,115.818277,20-09-12-16-14-31,A0:CF:5B:3D:7F:D0,47/70,-63,Unifi
1,-31.976884,115.818277,20-09-12-16-14-31,A0:CF:5B:3D:83:70,32/70,-78,Unifi
2,-31.976884,115.818277,20-09-12-16-14-31,A0:CF:5B:3D:7F:DF,31/70,-79,Unifi
3,-31.976884,115.818277,20-09-12-16-14-31,5C:50:15:E7:EA:30,38/70,-72,Unifi
4,-31.976834,115.818277,20-09-12-16-14-53,A0:CF:5B:3D:7F:D0,53/70,-57,Unifi
...,...,...,...,...,...,...,...
87,-31.976034,115.818277,20-09-12-16-17-46,64:AE:0C:C4:EC:00,33/70,-77,Unifi
88,-31.975984,115.818277,20-09-12-16-17-56,A0:CF:5B:3D:82:70,30/70,-80,Unifi
89,-31.975984,115.818277,20-09-12-16-17-56,A0:CF:5B:3F:8A:C0,28/70,-82,Unifi
90,-31.975984,115.818277,20-09-12-16-17-56,A0:CF:5B:3D:82:7F,27/70,-83,Unifi


In [17]:
test = wifi_data.loc[wifi_data['bssid'] == 'A0:CF:5B:3D:7F:D0']

for index, row in test.iterrows():
    print(row['lat'], row['lon'], row['rssi'])

-31.976884000000002 115.818277 -63
-31.976834000000004 115.818277 -57
-31.976784000000002 115.818277 -76
-31.976734000000004 115.818277 -76
-31.976684000000002 115.818277 -78
-31.976634000000004 115.818277 -78
-31.976584000000003 115.818277 -78


In [31]:
class WiFiMap:
    
    RADIUS_MODIFIER = 0.5
    CIRCLE_OPACITY = 0.5
    CIRCLE_WEIGHT = 1
    
    def __init__(self, wifi_df, length):
        """
        Initialises the WiFiMap class
        
        Parameters:
            wifi_df: pandas.dataframe
                The Dataframe with the WiFi measurements and locations
        """
        self.wifi_df = wifi_df
        self.current_df = wifi_df
        self.length = length
        
        # self.marker_layer = LayerGroup(name='markers')
        self.best_rssi_layer = LayerGroup(name='Best RSSI values')
        self.ap_layers = []
        # self.line_layer = LayerGroup(name='lines')
        
        self.map = Map(
            basemap = basemaps.Esri.WorldImagery,
            center = self.get_average_location(),
            zoom = 100
        )
        
        # self.map.add_layer(self.marker_layer)
        self.map.add_layer(self.best_rssi_layer)
        # self.map.add_layer(self.line_layer)
        self.map.add_control(FullScreenControl())
        self.map.add_control(LayersControl(position='topright'))
        
        
    def create_ap_layers(self):
        bssid_values = self.wifi_df['bssid'].to_list()
        
        for bssid in bssid_values:
            snapshot = wifi_data.loc[wifi_data['bssid'] == bssid]
            ap_layer_group = LayerGroup(name=bssid)
            
            for index, row in snapshot.iterrows():
                loc = (row['lat'], row['lon'])
                rssi =  row['rssi']
                
                colour = STRENGTH_COLOUR[classify_wifi_strength(rssi)]
                circle = Circle(
                    location = loc,
                    color = colour,
                    fill_color = colour,
                    radius = int(self.length*WiFiMap.RADIUS_MODIFIER),
                    opacity = WiFiMap.CIRCLE_OPACITY,
                    weight = WiFiMap.CIRCLE_WEIGHT
                )
                ap_layer_group.add_layer(circle)
                
            self.ap_layers.append(ap_layer_group)
            
        for ap_layer in self.ap_layers:
            self.map.add_layer(ap_layer)
                
            
        
    def create_map(self):
        best_strengths = self.current_df.groupby(['lat', 'lon'])['rssi'].max()
        
        for loc, rssi in best_strengths.items():
            colour = STRENGTH_COLOUR[classify_wifi_strength(rssi)]
            circle = Circle(
                location = loc,
                color = colour,
                fill_color = colour,
                radius = int(self.length*WiFiMap.RADIUS_MODIFIER),
                opacity = WiFiMap.CIRCLE_OPACITY,
                weight = WiFiMap.CIRCLE_WEIGHT
            )
            self.best_rssi_layer.add_layer(circle)
            
        self.create_ap_layers()
    
    def display_map(self):
        self.create_map()
        return self.map
        
    def get_average_location(self, bssid_filter=False):
        data_df = self.current_df
        return(data_df['lat'].mean(), data_df['lon'].mean())

In [32]:
wifi_map = WiFiMap(wifi_data, 5)
wifi_map.display_map()

Map(center=[-31.976441608695662, 115.81827700000001], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [70]:
scan_map = Map(
    basemap=basemaps.Esri.WorldImagery,
    center=(-31.97645513, 115.81831377),
    zoom=100
)

recorded_locations = []

marker = Marker(location=gps_data[0])

scan_map.add_layer(marker)
    
scan_map.add_control(FullScreenControl())

scan_map

Map(center=[-31.97645513, 115.81831377], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…

In [95]:
marker.location

[-31.980241664322044, 115.81775844097137]

In [94]:
save = '\n'.join([','.join([str(recorded_locations[i][j]) 
                     for j in range(2)]) for i in range(len(recorded_locations))])

In [92]:
LOCATION = 'test'
with open(GPS_FOLDER + LOCATION + '.csv', 'w') as f:
    f.write(save)

In [19]:
lats = pd.read_excel('./data/gps/coords.xlsx', sheet_name='whitfeld-1', usecols=["Latitude", "Longtitude"])

In [22]:
lats['time'] = pd.DataFrame(data=pd.read_csv('./data/wifi/whitfeld-1/complete.csv', names=['time', 'bssid', 'quality', 'rssi', 'essid'])['time'].unique(), columns=['time'])

In [23]:
lats

,Latitude,Longtitude,time
0,-31.976879,115.817472,20-09-12-15-48-16
1,-31.976829,115.817472,20-09-12-15-48-26
2,-31.976779,115.817472,20-09-12-15-48-36
3,-31.976729,115.817472,20-09-12-15-48-44
4,-31.976679,115.817472,20-09-12-15-48-58
5,-31.976629,115.817472,20-09-12-15-49-07
6,-31.976579,115.817472,20-09-12-15-49-18
7,-31.976529,115.817472,20-09-12-15-49-28
8,-31.976479,115.817472,20-09-12-15-49-42
9,-31.976429,115.817472,20-09-12-15-49-55
